In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
from acquire import get_zillow_data
from pyzipcode import ZipCodeDatabase
from geopy.geocoders import Nominatim
zcdb = ZipCodeDatabase()
geolocator = Nominatim(user_agent='geoaipExercises_1')

In [2]:
df = get_zillow_data()

In [3]:
df.shape

(38436, 13)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38436 entries, 0 to 38435
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    38436 non-null  float64
 1   bathroomcnt                   38436 non-null  float64
 2   calculatedfinishedsquarefeet  38323 non-null  float64
 3   taxvaluedollarcnt             38435 non-null  float64
 4   yearbuilt                     38300 non-null  float64
 5   taxamount                     38435 non-null  float64
 6   fips                          38436 non-null  float64
 7   regionidzip                   38409 non-null  float64
 8   regionidcity                  37713 non-null  float64
 9   lotsizesquarefeet             34084 non-null  float64
 10  latitude                      38436 non-null  float64
 11  longitude                     38436 non-null  float64
 12  transactiondate               38436 non-null  object 
dtypes

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,38436.0,3.013035e+00,1.030515,0.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.200000e+01
bathroomcnt,38436.0,2.274820e+00,0.952277,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.100000e+01
calculatedfinishedsquarefeet,38323.0,1.764788e+03,929.813109,2.400000e+02,1.174000e+03,1.527000e+03,2.085000e+03,1.724500e+04
taxvaluedollarcnt,38435.0,4.975543e+05,633989.590260,1.000000e+03,2.090130e+05,3.643870e+05,5.787730e+05,2.385837e+07
yearbuilt,38300.0,1.969476e+03,23.101822,1.878000e+03,1.954000e+03,1.970000e+03,1.987000e+03,2.016000e+03
taxamount,38435.0,6.056283e+03,7381.179259,1.992000e+01,2.724345e+03,4.476560e+03,7.001565e+03,2.767978e+05
fips,38436.0,6.049621e+03,21.048908,6.037000e+03,6.037000e+03,6.037000e+03,6.059000e+03,6.111000e+03
regionidzip,38409.0,9.662408e+04,4393.628442,9.598200e+04,9.620800e+04,9.641000e+04,9.698900e+04,3.996750e+05
regionidcity,37713.0,3.315465e+04,45124.269638,3.491000e+03,1.244700e+04,2.521800e+04,4.545700e+04,3.965560e+05
lotsizesquarefeet,34084.0,2.975629e+04,113092.569943,2.360000e+02,5.848000e+03,7.320000e+03,1.216925e+04,6.971010e+06


In [6]:
try:
    location_df = pd.read_csv('location_and_zillow_data.csv')
except:
    location_df = pd.DataFrame([{'latitude': None, 'longitude': None}])
    location_df.to_csv('location_and_zillow_data.csv', index=False)
unlocated_df = df[~((df.latitude.isin(location_df.latitude)) & (df.longitude.isin(location_df.longitude)))]
# population mapping 
popdf = pd.read_csv('2010_Census_Population_By_Zipcode.csv')

In [7]:
def cycle(unlocated_df, location_df):
    temp_df = unlocated_df.head(100)

    def location_lookup(latitude, longitude):
        lat = str(int(latitude))[:-6]+'.'+str(int(latitude))[-6:] 
        long = str(int(longitude))[:-6]+'.'+str(int(longitude))[-6:] 
        loc = lat + ',' + long
        location = geolocator.reverse(loc)
        location_dict = location.raw['address']
        return location_dict

    temp_df['location'] = temp_df.apply(lambda x: location_lookup(x.latitude, x.longitude), axis=1)
    locations = temp_df.location.apply(pd.Series)
    new_df = pd.merge(temp_df, locations, left_index=True, right_index=True)

    # Set index to zipcode
    tdf = popdf.set_index('zipcode')
    # make the zipcode a dict value to map
    pop = popdf.to_dict()
    # set population dict to be the population so it's formatted properly
    population = pop['population']
    # set population column to be the postcode to be mapped too
    new_df['population'] = new_df.postcode
    # replace the zipcode with the population information
    new_df.population.update(pd.Series(population))
    new_df.head()
    location_df = location_df.append(new_df)
    location_df.to_csv('location_and_zillow_data.csv', index=False)

In [8]:
try:
    location_df = pd.read_csv('location_and_zillow_data.csv')
except:
    location_df = pd.DataFrame([{'latitude': None, 'longitude': None}])
    location_df.to_csv('location_and_zillow_data.csv', index=False)
unlocated_df = df[~((df.latitude.isin(location_df.latitude)) & (df.longitude.isin(location_df.longitude)))]
# population mapping 
popdf = pd.read_csv('2010_Census_Population_By_Zipcode.csv')
location_df.head()

,latitude,longitude,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,regionidzip,...,leisure,man_made,retail,historic,aeroway,railway,commercial,place,office,craft
0,33922098,-117954970,3,2.0,1125.0,289445,1974.0,3490.16,6059,96186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34033180,-118355396,3,2.0,1316.0,205123,1923.0,2627.48,6037,95997.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34686163,-118113100,3,2.0,1458.0,136104,1970.0,2319.90,6037,97318.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34073222,-118390965,3,2.0,1766.0,810694,1980.0,9407.01,6037,96086.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33999877,-118291863,2,1.0,1421.0,35606,1911.0,543.69,6037,96018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# while unlocated_df.shape[0] != 0:
#     location_df = pd.read_csv('location_and_zillow_data.csv')
#     print(location_df.shape)
#     # Checks to see if the data has been found
#     unlocated_df = df[~((df.latitude.isin(location_df.latitude)) & (df.longitude.isin(location_df.longitude)))]
#     print(unlocated_df.shape[0])
#     location_df.head()
#     cycle(unlocated_df, location_df)
#     print('another cycle')

In [15]:
location_df.population

0             16769
1             29049
2             10372
3              5079
4             14649
            ...    
34239         90004
34240         92804
34241         92673
34242    91326-1729
34243    91311-4127
Name: population, Length: 34244, dtype: object